<a href="https://colab.research.google.com/github/LucasEPrz/APRENDIZAJE-POR-REFUERZO/blob/main/Lab2/Lab2_Perez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Actividad 1**

Crear tu propio entorno y entrenar agentes RL en el mismo. Analizar la convergencia con distintos algoritmos* (ej: PPO, DQN), resultados con distintas funciones de recompensa e híper-parámetros.

Algunas ideas:

Transformar GoLeftEnv en una grilla 2D, añadir paredes / trampas / agua.
Crear un entorno que juegue a algún juego como el ta-te-ti.
Crea un entorno totalmente nuevo que sea de tu interés!

# **Creación del entorno**

Definir la estructura del entorno

In [ ]:
import gym
from gym import spaces
import numpy as np

class CustomGridEnv(gym.Env):
    def __init__(self):
        super(CustomGridEnv, self).__init__()

        self.grid_size = 5  # Tamaño de la grilla (5x5)
        self.agent_pos = [0, 0]  # Posición inicial del agente (en la esquina superior izquierda)
        self.goal_pos = [4, 4]  # Posición del objetivo (en la esquina inferior derecha)

        # Definir la acción: [arriba, abajo, izquierda, derecha]
        self.action_space = spaces.Discrete(4)

        # Definir el espacio de observación: matriz 5x5, donde 0 = vacío, 1 = pared, 2 = trampa, 3 = objetivo
        self.observation_space = spaces.Box(low=0, high=3, shape=(self.grid_size, self.grid_size), dtype=int)

        # Mapa con paredes y trampas
        self.grid = np.zeros((self.grid_size, self.grid_size))
        self.grid[1, 1] = 1  # Pared en (1,1)
        self.grid[2, 2] = 2  # Trampa en (2,2)

    def reset(self):
        self.agent_pos = [0, 0]  # Reiniciar la posición del agente
        return self.grid  # Devuelve el estado inicial de la grilla

    def step(self, action):
        # Mover al agente en función de la acción
        if action == 0:  # Arriba
            self.agent_pos[0] = max(0, self.agent_pos[0] - 1)
        elif action == 1:  # Abajo
            self.agent_pos[0] = min(self.grid_size - 1, self.agent_pos[0] + 1)
        elif action == 2:  # Izquierda
            self.agent_pos[1] = max(0, self.agent_pos[1] - 1)
        elif action == 3:  # Derecha
            self.agent_pos[1] = min(self.grid_size - 1, self.agent_pos[1] + 1)

        # Verificar si el agente ha llegado al objetivo o ha caído en una trampa
        done = False
        reward = -1  # Penalización por cada paso
        if self.agent_pos == self.goal_pos:
            reward = 10  # Recompensa por llegar al objetivo
            done = True
        elif self.grid[self.agent_pos[0], self.agent_pos[1]] == 2:  # Si cae en una trampa
            reward = -10  # Recompensa negativa por caer en una trampa
            done = True

        return self.grid, reward, done, {}

    def render(self):
        # Imprimir la grilla para visualizar el entorno
        grid_copy = self.grid.copy()
        grid_copy[self.agent_pos[0], self.agent_pos[1]] = 5  # Representar al agente con un 5
        print(grid_copy)


# **Entrenar el agente con un algoritmo de RL**

In [ ]:
!pip install 'shimmy>=2.0'

In [ ]:
!pip install stable-baselines3[extra]

# Entrenar el agente con PPO

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

# Crear el entorno
env = CustomGridEnv()

# Vectorizar el entorno
env = DummyVecEnv([lambda: env])

# Crear y entrenar el agente PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Using cpu device
-----------------------------
| time/              |      |
|    fps             | 1026 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 668        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.01902721 |
|    clip_fraction        | 0.31       |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.37      |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 13         |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0263    |
|    value_loss           | 75.7       |
----------------------------------------
-----------------------------------

Analizar la convergencia

In [ ]:
model = PPO("MlpPolicy", env, verbose=1, learning_rate=0.001, n_steps=256, gamma=0.99, ent_coef=0.01)

Using cpu device


Evaluación del agente

In [ ]:
state = env.reset()
for _ in range(1000):
    action, _states = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
    if done:
        break

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


# Probar con otros algoritmos (DQN)

In [ ]:
from stable_baselines3 import DQN

# Entrenar con DQN
model_dqn = DQN("MlpPolicy", env, verbose=1)
model_dqn.learn(total_timesteps=10000)

Using cpu device
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.578    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1391     |
|    time_elapsed     | 0        |
|    total_timesteps  | 444      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.687    |
|    n_updates        | 85       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 985      |
|    time_elapsed     | 2        |
|    total_timesteps  | 2528     |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.000332 |
|    n_updates        | 606      |
----------------------------------
----------------------------------
| rollout/            |          |
|  

Graficar la convergencia

In [ ]:
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.callbacks import EvalCallback

eval_callback = EvalCallback(env, best_model_save_path='./logs/', log_path='./logs/', eval_freq=5000, deterministic=True, render=False)

# Entrenar el modelo con el callback
model.learn(total_timesteps=10000, callback=eval_callback)



-----------------------------
| time/              |      |
|    fps             | 1187 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 256  |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 846         |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 512         |
| train/                  |             |
|    approx_kl            | 0.015029857 |
|    clip_fraction        | 0.187       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.02       |
|    explained_variance   | 0           |
|    learning_rate        | 0.001       |
|    loss                 | 28.4        |
|    n_updates            | 70          |
|    policy_gradient_loss | -0.0203     |
|    value_loss           | 57.1        |
-----------------------------------------
----------------------------------